In [7]:
import os
import sys
import pandas as pd
from sqlalchemy import create_engine
import backtrader as bt
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_path)
print(sys.path)


from scripts.backtest_runner import fetch_data, RsiBollingerBandsStrategy, SimpleMovingAverageStrategy,RefinedSMAStrategy,run_backtest

rds_host = os.getenv('PG_HOST')
rds_port = os.getenv('PG_PORT')
rds_db = os.getenv('PG_DATABASE')
rds_user = os.getenv('PG_USER')
rds_password = os.getenv('PG_PASSWORD')

engine = create_engine(f'postgresql+psycopg2://{rds_user}:{rds_password}@{rds_host}:{rds_port}/{rds_db}')

['/home/air/crypto-trading-backtesting/notebooks', '/home/air/.pyenv/versions/3.10.2/lib/python310.zip', '/home/air/.pyenv/versions/3.10.2/lib/python3.10', '/home/air/.pyenv/versions/3.10.2/lib/python3.10/lib-dynload', '', '/home/air/crypto-trading-backtesting/.crypto/lib/python3.10/site-packages', '/home/air/crypto-trading-backtesting', '/home/air/crypto-trading-backtesting']


## Parameters

In [2]:
symbol = 'BTC/USDT'
start_date = '2023-06-20'
end_date = '2024-06-20'

## Fetch data and display first few rows

In [3]:
data = fetch_data(symbol, start_date, end_date)
data.head()

Executing query:

        SELECT timestamp AS date, open AS open, high AS high, low AS low, close AS close, volume AS volume 
        FROM public."ohlcv_BTC_USDT"
        WHERE timestamp >= '2023-06-20' AND timestamp <= '2024-06-20';
    

Fetched data:
        date      open      high       low     close        volume
0 2023-06-20  26844.35  28402.74  26652.00  28307.99   69666.95525
1 2023-06-21  28308.00  30800.00  28257.99  29993.89  108926.40412
2 2023-06-22  29993.89  30500.00  29525.61  29884.92   59054.56460
3 2023-06-23  29884.92  31431.94  29800.00  30688.50   73931.89635
4 2023-06-24  30688.51  30800.00  30250.00  30527.43   30513.30135



,Open,High,Low,Close,Volume
date,,,,,
2023-06-20,26844.35,28402.74,26652.00,28307.99,69666.95525
2023-06-21,28308.00,30800.00,28257.99,29993.89,108926.40412
2023-06-22,29993.89,30500.00,29525.61,29884.92,59054.56460
2023-06-23,29884.92,31431.94,29800.00,30688.50,73931.89635
2023-06-24,30688.51,30800.00,30250.00,30527.43,30513.30135


## Run backtest for SimpleMovingAverageStrategy

In [4]:
result = run_backtest(SimpleMovingAverageStrategy, symbol, start_date, end_date)
print(f"Starting Portfolio Value: {result['Starting Portfolio Value']}")
print(f"Ending Portfolio Value: {result['Ending Portfolio Value']}")
print(result)

Executing query:

        SELECT timestamp AS date, open AS open, high AS high, low AS low, close AS close, volume AS volume 
        FROM public."ohlcv_BTC_USDT"
        WHERE timestamp >= '2023-06-20' AND timestamp <= '2024-06-20';
    

Fetched data:
        date      open      high       low     close        volume
0 2023-06-20  26844.35  28402.74  26652.00  28307.99   69666.95525
1 2023-06-21  28308.00  30800.00  28257.99  29993.89  108926.40412
2 2023-06-22  29993.89  30500.00  29525.61  29884.92   59054.56460
3 2023-06-23  29884.92  31431.94  29800.00  30688.50   73931.89635
4 2023-06-24  30688.51  30800.00  30250.00  30527.43   30513.30135

Starting Portfolio Value: 100000.00
2023-07-23, Close, 30083.75
2023-07-24, Close, 29176.50
2023-07-25, Close, 29228.91
2023-07-26, Close, 29351.96
2023-07-27, Close, 29222.78
2023-07-28, Close, 29314.14
2023-07-29, Close, 29352.90
2023-07-30, Close, 29281.09
2023-07-31, Close, 29232.25
2023-08-01, Close, 29705.99
2023-08-01, BUY CREATE, 297

<IPython.core.display.Javascript object>

Starting Portfolio Value: 100000
Ending Portfolio Value: 102562.00163999997
{'Starting Portfolio Value': 100000, 'Ending Portfolio Value': 102562.00163999997, 'Sharpe Ratio': 0.24330544385607064, 'Max Drawdown': 21.43399023145571, 'Total Trades': 27, 'Winning Trades': 4, 'Losing Trades': 23, 'Total Return': 0.02529732376448946}


In [5]:
import matplotlib.pyplot as plt
# Plot Close Price with SMA

plt.figure(figsize=(14, 7))
plt.plot(data.index, data['Close'], label='Close Price')
plt.title('BTC/USDT Close Price')
plt.xlabel('Date')
plt.ylabel('Price (USDT)')
plt.legend()
plt.show()
     

ValueError: Multi-dimensional indexing (e.g. `obj[:, None]`) is no longer supported. Convert to a numpy array before indexing instead.

Executing query:

        SELECT timestamp AS date, open AS open, high AS high, low AS low, close AS close, volume AS volume 
        FROM public."ohlcv_BTC_USDT"
        WHERE timestamp >= '2023-06-20' AND timestamp <= '2024-06-20';
    

Fetched data:
        date      open      high       low     close        volume
0 2023-06-20  26844.35  28402.74  26652.00  28307.99   69666.95525
1 2023-06-21  28308.00  30800.00  28257.99  29993.89  108926.40412
2 2023-06-22  29993.89  30500.00  29525.61  29884.92   59054.56460
3 2023-06-23  29884.92  31431.94  29800.00  30688.50   73931.89635
4 2023-06-24  30688.51  30800.00  30250.00  30527.43   30513.30135

Starting Portfolio Value: 100000.00
2023-08-08, Close, 29770.42
2023-08-09, Close, 29581.99
2023-08-10, Close, 29455.75
2023-08-11, Close, 29426.03
2023-08-12, Close, 29430.17
2023-08-13, Close, 29303.84
2023-08-14, Close, 29430.93
2023-08-15, Close, 29200.00
2023-08-16, Close, 28730.51
2023-08-17, Close, 26623.41
2023-08-18, Close, 26054.00

<IPython.core.display.Javascript object>

Starting Portfolio Value: 100000
Ending Portfolio Value: 123561.09554000002
{'Starting Portfolio Value': 100000, 'Ending Portfolio Value': 123561.09554000002, 'Sharpe Ratio': 3.360025269585281, 'Max Drawdown': 13.10642999421397, 'Total Trades': 3, 'Winning Trades': 2, 'Losing Trades': 0, 'Total Return': 0.21156554848462109}


## Strategy and Indicators
### Strategy:
- The strategy implemented is based on a Simple Moving Average (SMA) crossover:

    - Buy Condition: The strategy attempts to buy when the closing price of the asset is above the SMA (15-period).
    - Sell Condition: The strategy attempts to sell when the closing price of the asset is below the SMA (15-period).

### Indicators:    

- Simple Moving Average (SMA): A 15-period SMA is used to determine the buy/sell signals.
- Additional indicators are included for plotting and analysis but are not used in the buy/sell decision logic:
    - Exponential Moving Average (EMA): 25-period.
    - Weighted Moving Average (WMA): 25-period.
    - Stochastic Slow: Stochastic indicator to measure momentum.
    - MACD Histogram: Moving Average Convergence Divergence for trend-following.
    - Relative Strength Index (RSI): To measure the speed and change of price movements.
    - ATR (Average True Range): Measures volatility.

In [11]:
result = run_backtest(RefinedSMAStrategy, symbol, start_date, end_date)
print(f"Starting Portfolio Value: {result['Starting Portfolio Value']}")
print(f"Ending Portfolio Value: {result['Ending Portfolio Value']}")
print(result)

Executing query:

        SELECT timestamp AS date, open AS open, high AS high, low AS low, close AS close, volume AS volume 
        FROM public."ohlcv_BTC_USDT"
        WHERE timestamp >= '2023-06-20' AND timestamp <= '2024-06-20';
    

Fetched data:
        date      open      high       low     close        volume
0 2023-06-20  26844.35  28402.74  26652.00  28307.99   69666.95525
1 2023-06-21  28308.00  30800.00  28257.99  29993.89  108926.40412
2 2023-06-22  29993.89  30500.00  29525.61  29884.92   59054.56460
3 2023-06-23  29884.92  31431.94  29800.00  30688.50   73931.89635
4 2023-06-24  30688.51  30800.00  30250.00  30527.43   30513.30135

Starting Portfolio Value: 100000.00
2023-08-08, Close, 29770.42
2023-08-09, Close, 29581.99
2023-08-10, Close, 29455.75
2023-08-11, Close, 29426.03
2023-08-12, Close, 29430.17
2023-08-13, Close, 29303.84
2023-08-14, Close, 29430.93
2023-08-15, Close, 29200.00
2023-08-16, Close, 28730.51
2023-08-17, Close, 26623.41
2023-08-18, Close, 26054.00

## Note
- The RefinedSMAStrategy uses both a short-term and a long-term moving average to generate buy and sell signals, which is a common and effective approach in trading strategies.

# Cryptocurrency Trading Backtesting for RSI and Bollinger Bands strategy
This notebook demonstrates the process of backtesting a cryptocurrency trading strategy using historical data. We will use the RSI and Bollinger Bands strategy to trade Bitcoin (BTC) against Tether (USDT).

# Run backtest

In [14]:
result = run_backtest(RsiBollingerBandsStrategy, symbol, start_date, end_date)
print(result)

Executing query:

        SELECT timestamp AS date, open AS open, high AS high, low AS low, close AS close, volume AS volume 
        FROM public."ohlcv_BTC_USDT"
        WHERE timestamp >= '2023-06-20' AND timestamp <= '2024-06-20';
    

Fetched data:
        date      open      high       low     close        volume
0 2023-06-20  26844.35  28402.74  26652.00  28307.99   69666.95525
1 2023-06-21  28308.00  30800.00  28257.99  29993.89  108926.40412
2 2023-06-22  29993.89  30500.00  29525.61  29884.92   59054.56460
3 2023-06-23  29884.92  31431.94  29800.00  30688.50   73931.89635
4 2023-06-24  30688.51  30800.00  30250.00  30527.43   30513.30135

Starting Portfolio Value: 100000.00
Ending Portfolio Value: 100479.17


<IPython.core.display.Javascript object>

{'Starting Portfolio Value': 100000, 'Ending Portfolio Value': 100479.17268, 'Sharpe Ratio': -3.173860663341704, 'Max Drawdown': 2.5275415753733292, 'Total Trades': 1, 'Winning Trades': 1, 'Losing Trades': 0, 'Total Return': 0.004780283019556264}


# Fetch data and display first few rows

In [15]:
data = fetch_data(symbol, start_date, end_date)
data.head()

Executing query:

        SELECT timestamp AS date, open AS open, high AS high, low AS low, close AS close, volume AS volume 
        FROM public."ohlcv_BTC_USDT"
        WHERE timestamp >= '2023-06-20' AND timestamp <= '2024-06-20';
    

Fetched data:
        date      open      high       low     close        volume
0 2023-06-20  26844.35  28402.74  26652.00  28307.99   69666.95525
1 2023-06-21  28308.00  30800.00  28257.99  29993.89  108926.40412
2 2023-06-22  29993.89  30500.00  29525.61  29884.92   59054.56460
3 2023-06-23  29884.92  31431.94  29800.00  30688.50   73931.89635
4 2023-06-24  30688.51  30800.00  30250.00  30527.43   30513.30135



,Open,High,Low,Close,Volume
date,,,,,
2023-06-20,26844.35,28402.74,26652.00,28307.99,69666.95525
2023-06-21,28308.00,30800.00,28257.99,29993.89,108926.40412
2023-06-22,29993.89,30500.00,29525.61,29884.92,59054.56460
2023-06-23,29884.92,31431.94,29800.00,30688.50,73931.89635
2023-06-24,30688.51,30800.00,30250.00,30527.43,30513.30135


## Plotting the Historical Data
Let's visualize the historical price data to understand the market trends.

In [7]:
plt.figure(figsize=(14, 7))
plt.plot(data.index, data['Close'], label='Close Price')
plt.title('BTC/USDT Close Price')
plt.xlabel('Date')
plt.ylabel('Price (USDT)')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

## Running the Backtest
Now, let's run the backtest using the RSI and Bollinger Bands strategy.


In [16]:
# Run backtest with RsiBollingerBandsStrategy
result = run_backtest(RsiBollingerBandsStrategy, symbol, start_date, end_date)
print(f"Starting Portfolio Value: {result['Starting Portfolio Value']}")
print(f"Ending Portfolio Value: {result['Ending Portfolio Value']}")
print(result)

Executing query:

        SELECT timestamp AS date, open AS open, high AS high, low AS low, close AS close, volume AS volume 
        FROM public."ohlcv_BTC_USDT"
        WHERE timestamp >= '2023-06-20' AND timestamp <= '2024-06-20';
    

Fetched data:
        date      open      high       low     close        volume
0 2023-06-20  26844.35  28402.74  26652.00  28307.99   69666.95525
1 2023-06-21  28308.00  30800.00  28257.99  29993.89  108926.40412
2 2023-06-22  29993.89  30500.00  29525.61  29884.92   59054.56460
3 2023-06-23  29884.92  31431.94  29800.00  30688.50   73931.89635
4 2023-06-24  30688.51  30800.00  30250.00  30527.43   30513.30135

Starting Portfolio Value: 100000.00
Ending Portfolio Value: 100479.17


<IPython.core.display.Javascript object>

Starting Portfolio Value: 100000
Ending Portfolio Value: 100479.17268
{'Starting Portfolio Value': 100000, 'Ending Portfolio Value': 100479.17268, 'Sharpe Ratio': -3.173860663341704, 'Max Drawdown': 2.5275415753733292, 'Total Trades': 1, 'Winning Trades': 1, 'Losing Trades': 0, 'Total Return': 0.004780283019556264}


## Visualizing the Strategy
We will plot the closing prices along with the RSI and Bollinger Bands to visualize the strategy's performance.


In [18]:
class PlotStrategy(bt.Strategy):
    params = (
        ('rsi_period', 14),
        ('bb_period', 20),
        ('bb_dev', 2),
    )

    def __init__(self):
        self.rsi = bt.indicators.RelativeStrengthIndex(period=self.params.rsi_period)
        self.bbands = bt.indicators.BollingerBands(period=self.params.bb_period, devfactor=self.params.bb_dev)
        self.data_close = self.data.close

    def next(self):
        pass

    def plot(self):
        plt.figure(figsize=(14, 7))
        plt.plot(self.data.datetime.array, self.data_close.array, label='Close Price')
        plt.plot(self.data.datetime.array, self.bbands.lines.bot.array, linestyle='dotted', label='Lower Bollinger Band')
        plt.plot(self.data.datetime.array, self.bbands.lines.mid.array, linestyle='dotted', label='Middle Bollinger Band')
        plt.plot(self.data.datetime.array, self.bbands.lines.top.array, linestyle='dotted', label='Upper Bollinger Band')
        plt.title('BTC/USDT with Bollinger Bands')
        plt.xlabel('Date')
        plt.ylabel('Price (USDT)')
        plt.legend()
        plt.show()

        plt.figure(figsize=(14, 3))
        plt.plot(self.data.datetime.array, self.rsi.array, label='RSI')
        plt.axhline(30, linestyle='dotted', color='red', label='Oversold')
        plt.axhline(70, linestyle='dotted', color='red', label='Overbought')
        plt.title('RSI')
        plt.xlabel('Date')
        plt.ylabel('RSI')
        plt.legend()
        plt.show()

# Running the strategy for plotting
data_feed = bt.feeds.PandasData(dataname=data)

cerebro = bt.Cerebro()
cerebro.addstrategy(PlotStrategy)
cerebro.adddata(data_feed)

cerebro.run()[0].plot()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>